In [ ]:
!pip install jmespath

In [ ]:
import json
import random
from pathlib import Path
from lxml import etree
import jmespath
from jsonschema import validate, ValidationError

In [ ]:
class FilmsCatalog:
    """Класс для работы с каталогом фильмов"""

    def __init__(self, base_path="."):
        self.base_path = Path(base_path)
        self.xml_tree = None
        self.json_data = None

    def load_xml(self, filename="films.xml"):
        """Загрузка XML файла"""
        try:
            self.xml_tree = etree.parse(str(self.base_path / filename))
            print(f"XML файл '{filename}' загружен")
            return True
        except Exception as e:
            print(f"Ошибка загрузки XML: {e}")
            return False

    def load_json(self, filename="films.json"):
        """Загрузка JSON файла"""
        try:
            with open(self.base_path / filename, 'r', encoding='utf-8') as f:
                self.json_data = json.load(f)
            print(f"JSON файл '{filename}' загружен")
            return True
        except Exception as e:
            print(f"Ошибка загрузки JSON: {e}")
            return False


    def validate_dtd(self, dtd_file="films.dtd"):
        """Валидация XML с DTD"""
        try:
            dtd = etree.DTD(str(self.base_path / dtd_file))
            if dtd.validate(self.xml_tree):
                print("XML валиден по DTD")
                return True
            print(f"Ошибки DTD: {dtd.error_log}")
            return False
        except Exception as e:
            print(f"Ошибка: {e}")
            return False

    def validate_xsd(self, xsd_file="films.xsd"):
        """Валидация XML с XSD"""
        try:
            schema = etree.XMLSchema(etree.parse(str(self.base_path / xsd_file)))
            if schema.validate(self.xml_tree):
                print("XML валиден по XSD")
                return True
            print(f"Ошибки XSD: {schema.error_log}")
            return False
        except Exception as e:
            print(f"Ошибка: {e}")
            return False

    def validate_json_schema(self, schema_file="films_schema.json"):
        """Валидация JSON"""
        try:
            with open(self.base_path / schema_file, 'r', encoding='utf-8') as f:
                schema = json.load(f)
            validate(self.json_data, schema)
            print("JSON валиден по JSON Schema")
            return True
        except ValidationError as e:
            print(f"Ошибка валидации: {e.message}")
            return False


    def transform(self, xslt_file, output_file):
        """XSLT преобразование"""
        try:
            xslt = etree.XSLT(etree.parse(str(self.base_path / xslt_file)))
            result = str(xslt(self.xml_tree))
            with open(self.base_path / output_file, 'w', encoding='utf-8') as f:
                f.write(result)
            print(f"{xslt_file} -> {output_file}")
            return result
        except Exception as e:
            print(f"Ошибка XSLT: {e}")
            return None


    def xpath_by_genre(self, genre):
        """a) Фильмы указанного жанра"""
        results = self.xml_tree.xpath(f"//film[genres/genre = '{genre}']")
        print(f"\n Фильмы жанра '{genre}' ({len(results)} шт.):")
        for f in results:
            print(f" {f.find('title').text} ({f.find('directors/director').text})")
        return results

    def xpath_large_cast(self):
        """b) Фильмы с актёрским составом > 5"""
        results = self.xml_tree.xpath("//film[count(cast/actor) > 5]")
        print(f"\n Фильмы с большим актёрским составом ({len(results)} шт.):")
        for f in results:
            count = len(f.find('cast').findall('actor'))
            print(f"{f.find('title').text} — {count} актёров")
        return results

    def xpath_director_actor(self, person):
        """c) Фильмы где человек — режиссёр и актёр"""
        results = self.xml_tree.xpath(
            f"//film[directors/director = '{person}' and cast/actor = '{person}']"
        )
        print(f"\n Фильмы где '{person}' — режиссёр и актёр:")
        if results:
            for f in results:
                print(f"{f.find('title').text}")
        else:
            print("  (не найдено)")
        return results

    def xpath_watchlist(self, genre, count):
        """d) Случайный список просмотра"""
        all_films = self.xml_tree.xpath(f"//film[genres/genre = '{genre}']")
        selected = random.sample(all_films, min(count, len(all_films)))
        print(f"Список просмотра ({len(selected)} фильмов жанра '{genre}'):")
        for i, f in enumerate(selected, 1):
            print(f"  {i}. {f.find('title').text}")
        return selected

    def xpath_custom(self):
        """e) Немецкоязычные фильмы с количеством зрителей > 1 млн и жанром Драма"""
        results = self.xml_tree.xpath(
            "//film[contains(originalLanguage, 'Немецкий') and viewerCount > 1000000 and genres/genre = 'Драма']"
        )
        print(f"Немецкоязычные драмы с аудиторией > 1 млн ({len(results)} шт.):")
        for f in results:
            viewers = int(f.find('viewerCount').text)
            print(f" {f.find('title').text} — {viewers:,} зрителей")
        return results


    def json_by_genre(self, genre):
        """a) Фильмы указанного жанра (JSON)"""
        results = jmespath.search(f"films[?contains(genres, '{genre}')]", self.json_data) or []
        print(f"\n [JSON] Фильмы жанра '{genre}' ({len(results)} шт.):")
        for f in results:
            print(f"  • {f['title']} ({f['directors'][0]})")
        return results

    def json_large_cast(self):
        """b) Фильмы с актёрским составом > 5 (JSON)"""
        results = jmespath.search("films[?length(cast) > `5`]", self.json_data) or []
        print(f"\n [JSON] Фильмы с большим актёрским составом ({len(results)} шт.):")
        for f in results:
            print(f"{f['title']} — {len(f['cast'])} актёров")
        return results

    def json_director_actor(self, person):
        """c) Фильмы где человек — режиссёр и актёр (JSON)"""
        results = jmespath.search(
            f"films[?contains(directors, '{person}') && contains(cast, '{person}')]",
            self.json_data
        ) or []
        print(f"\n [JSON] Фильмы где '{person}' — режиссёр и актёр:")
        if results:
            for f in results:
                print(f"  • {f['title']}")
        else:
            print("  (не найдено)")
        return results

    def json_watchlist(self, genre, count):
        """d) Случайный список просмотра (JSON)"""
        all_films = jmespath.search(f"films[?contains(genres, '{genre}')]", self.json_data) or []
        selected = random.sample(all_films, min(count, len(all_films)))
        print(f"\n [JSON] Список просмотра ({len(selected)} фильмов жанра '{genre}'):")
        for i, f in enumerate(selected, 1):
            print(f"  {i}. {f['title']}")
        return selected

    def json_custom(self):
        """e) Фильмы 18+ со сборами > 100 млн (JSON)"""
        results = jmespath.search(
            "films[?ageRating == '18+' && boxOffice.amount > `100000000`]",
            self.json_data
        ) or []
        print(f"\n [JSON] Фильмы 18+ со сборами > 100 млн ({len(results)} шт.):")
        for f in results:
            print(f"  • {f['title']} — {f['boxOffice']['amount']:,} {f['boxOffice']['currency']}")
        return results

In [ ]:
def main():

    catalog = FilmsCatalog()

    # Загрузка и валидация XML
    print("ЗАГРУЗКА И ВАЛИДАЦИЯ")
    catalog.load_xml()
    catalog.validate_dtd()
    catalog.validate_xsd()

    # XSLT преобразования
    print("XSLT ПРЕОБРАЗОВАНИЯ")
    catalog.transform("films_to_txt.xslt", "output.txt")
    catalog.transform("films_to_html.xslt", "output.html")
    catalog.transform("films_to_json.xslt", "films.json")

    # Загрузка и валидация JSON
    catalog.load_json()
    catalog.validate_json_schema()

    # XPath запросы
    print("XPATH ЗАПРОСЫ")
    catalog.xpath_by_genre("Драма")
    catalog.xpath_large_cast()
    catalog.xpath_director_actor("Квентин Тарантино")
    catalog.xpath_watchlist("Триллер", 3)
    catalog.xpath_custom()

    # JSON запросы
    print("JSON ЗАПРОСЫ")
    catalog.json_by_genre("Ужасы")
    catalog.json_large_cast()
    catalog.json_director_actor("Квентин Тарантино")
    catalog.json_watchlist("Драма", 4)
    catalog.json_custom()



In [ ]:
if __name__ == "__main__":
    main()

ЗАГРУЗКА И ВАЛИДАЦИЯ
XML файл 'films.xml' загружен
Ошибки DTD: films.xml:3:0:ERROR:VALID:DTD_ATTRIBUTE_VALUE: Syntax of value for attribute id of film is not valid
films.xml:28:0:ERROR:VALID:DTD_ATTRIBUTE_VALUE: Syntax of value for attribute id of film is not valid
films.xml:54:0:ERROR:VALID:DTD_ATTRIBUTE_VALUE: Syntax of value for attribute id of film is not valid
films.xml:82:0:ERROR:VALID:DTD_ATTRIBUTE_VALUE: Syntax of value for attribute id of film is not valid
films.xml:110:0:ERROR:VALID:DTD_ATTRIBUTE_VALUE: Syntax of value for attribute id of film is not valid
films.xml:137:0:ERROR:VALID:DTD_ATTRIBUTE_VALUE: Syntax of value for attribute id of film is not valid
films.xml:164:0:ERROR:VALID:DTD_ATTRIBUTE_VALUE: Syntax of value for attribute id of film is not valid
films.xml:194:0:ERROR:VALID:DTD_ATTRIBUTE_VALUE: Syntax of value for attribute id of film is not valid
films.xml:221:0:ERROR:VALID:DTD_ATTRIBUTE_VALUE: Syntax of value for attribute id of film is not valid
films.xml:248:0